# Basic Use Case: Chat with memory and personality

In [20]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

groq_api_key = os.environ["GROQ_API_KEY"]
langchain_key = os.environ["LANGCHAIN_API_KEY"] 
os.environ["LANGCHAIN_TRACING_V2"] = "true"


# Setup LLM to use, llama3 using Groq

In [21]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0, # no hallucination 
    max_tokens=None,
    timeout=None,
    max_retries=2,
    verbose=True
    # other params...
)

In [22]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

In [23]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who speaks in {language}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | llm | StrOutputParser()

runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)



In [24]:
runnable_with_history.invoke(
    {'language': 'spanish', 'input':'Hi I am Luis, I am 30 years old, I want to ask you how tall are mexicos most tall building'}
    ,
    config={"configurable": {"user_id": "456", "session_id": "1"}}

)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'¡Hola Luis! ¡No te preocupes, ya sabes la respuesta! El edificio más alto de México es el Torre Latinoamericana, que mide 182 metros de altura.'

In [25]:
runnable_with_history.invoke(
    {'language': 'spanish', 'input':'I forgot my name, whats my name'}
    ,
    config={"configurable": {"user_id": "456", "session_id": "1"}}
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'¡No te preocupes, Luis!'

In [26]:
runnable_with_history.invoke(
    {'language': 'spanish', 'input':'what I asked before?'}
    ,
    config={"configurable": {"user_id": "456", "session_id": "1"}}
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'¡Ah! Preguntaste sobre el edificio más alto de México y también te preguntaste qué era tu nombre.'

In [27]:
runnable_with_history.invoke(
    {'language': 'spanish', 'input':'how old am I?'}
    ,
    config={"configurable": {"user_id": "456", "session_id": "1"}}
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'¡Tú tienes 30 años, Luis!'